# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


All deep_gemm operations loaded successfully!


`torch_dtype` is deprecated! Use `dtype` instead!


/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
/usr/local/lib/python3.10/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


`torch_dtype` is deprecated! Use `dtype` instead!
[2025-09-27 17:25:00] `torch_dtype` is deprecated! Use `dtype` instead!


...

Capturing batches (bs=40 avail_mem=68.73 GB):  55%|█████▌    | 11/20 [00:01<00:00, 11.68it/s]

Capturing batches (bs=16 avail_mem=68.71 GB):  65%|██████▌   | 13/20 [00:01<00:00, 13.32it/s]

Capturing batches (bs=2 avail_mem=68.67 GB):  85%|████████▌ | 17/20 [00:01<00:00, 14.12it/s]

Capturing batches (bs=1 avail_mem=68.67 GB):  95%|█████████▌| 19/20 [00:01<00:00, 11.73it/s]

Capturing batches (bs=1 avail_mem=68.67 GB): 100%|██████████| 20/20 [00:01<00:00, 10.43it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Karin. I am a 14-year-old girl living in the United States. I am very happy because I get to learn new things about the world. I really enjoy traveling, and I want to travel to different places in the world. The interesting part of my life is my school life. When I go to school, I have to learn a lot of things. It is very busy and full of people. I like to stay with my friends and do my homework. My parents always encourage me to learn. I like to have fun. I go out with my friends and have ice cream. Sometimes I get so much fun
Prompt: The president of the United States is
Generated text:  trying to select a new logo for the country. The logo design must include a rectangle and a circle, with the rectangle being 25 inches wide and 30 inches tall. The circle must have an area of 36 square inches. What is the area of the rectangle and the circle?
To determine the area of the rectangle, we use the formula for the area of a rectangle, which is giv

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about your career. What can you tell me about yourself? [Name] is a [job title] at [company name], and I'm excited to meet you and learn more about your career. What can you tell me about yourself? [Name] is a [job title] at [company name], and I'm excited to meet you and learn more about your career. What can you tell me about yourself? [Name] is a [job title] at [company name], and I'm excited to

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, which is known for its iconic landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. It is also home to the French Parliament and the French National Museum of Modern Art. Paris is a bustling city with a rich cultural heritage and is a popular tourist destination. It is the capital of France and the largest city in the European Union. 

Paris is known for its rich history, including the Roman Empire, the French Revolution, and the French Revolution. It is also famous for its fashion, art, and cuisine. The city is home to many famous landmarks and attractions, including the Louvre

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies will continue to improve and become more integrated into our daily lives, from self-driving cars to personalized medicine. AI will also continue to be used for tasks that require human-like intelligence, such as language translation and emotional intelligence. However, there are also potential risks and challenges associated with AI, such as job displacement and ethical concerns. As we continue to develop and integrate AI into our daily lives, it is important to carefully consider the potential benefits and risks and work to ensure that AI is used in a responsible and ethical manner.



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a [age] year old [occupation]. I'm an [experience] that I'm always [achievement]. I'm passionate about [motivational goal]. I'm excited to share some [additionnal info] with you today. [If you have any other information that you think is helpful for me to know] Let's get started! 📸✨

Note: I'll try to keep the self-introduction brief, neutral, and brief in length. I'll use words that are appropriate for a fictional character and add an element of authenticity to the introduction. Let's get started!

Hello

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, an iconic city with a rich history and culture that has become a global cultural and political hub. Located in the south of the country, Paris is known for its iconic landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathed

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Your

 Name

].

...

 to

 play

 a

 growing

 role

In [6]:
llm.shutdown()